# Iterators, generator expressions, and generators

## Iterators

In [4]:
nums = [1,2,3]
iter(nums)

In [5]:
nums.__iter__()

In [6]:
nums.__reversed__()

In [7]:
it = iter(nums)
next(it)

1

In [8]:
next(it)

2

In [9]:
next(it)

3

In [10]:
# When used in a loop, StopIteration is swallowed and 
# causes the loop to finish. But with explicit invocation, 
# we can see that once the iterator is exhausted, 
# accessing it raises an exception
try:
    next(it)
except Exception:
    pass
finally:
    pass


## Generator expressions

Generator expression is the basis of __list comprehension__. Generator expression must always be enclosed in parentheses or an expression. If round parentheses are used, then a generator iterator is created. If rectangular parentheses are used, the process is short-circuited and we get a list.

In [11]:
(i for i in nums)

<generator object <genexpr> at 0x1101cad58>

In [12]:
[i for i in nums]

[1, 2, 3]

In [13]:
list(i for i in nums)

[1, 2, 3]

In [14]:
# similarly applied to set and dictionary comprehensions

In [15]:
myset = {i for i in nums}
myset

{1, 2, 3}

In [16]:
type(myset)

set

In [17]:
mydict = {i:i**2 for i in nums}
mydict

{1: 1, 2: 4, 3: 9}

In [18]:
type(mydict)

dict

## Generator function

Generator functions are marked with the keyword __yield__.

In [19]:
def f():
    yield 1
    yield 2
    
f()

<generator object f at 0x10fc32e60>

In [20]:
gen = f()

In [21]:
next(gen)

1

In [22]:
next(gen)

2

In [23]:
try:
    next(gen)
except Exception:
    pass

In [24]:
def f():
    print('-- start --')
    yield 3
    print('-- middle --')
    yield 4
    print('-- finished --')
    
gen = f() # nothing is printed! the first action will take place after 'next'

In [25]:
next(gen)

-- start --


3

In [26]:
next(gen)

-- middle --


4

In [27]:
try:
    next(gen)
except Exception:
    pass

-- finished --


## Bidirectional communication

In [28]:
import itertools
def g():
    print('-- start --')
    for i in itertools.count():
        print('-- yielding %i --' % i)
        try:
            ans = yield i
        except GeneratorExit:
            print('-- closing --')
            raise
        except Exception as e:
            print('--yield raised %r--' % e)
        else:
            print('--yield returned %s--' % ans)
            
it = g()

In [29]:
next(it)

-- start --
-- yielding 0 --


0

In [30]:
it.send(11)

--yield returned 11--
-- yielding 1 --


1

In [31]:
it.throw(IndexError)

--yield raised IndexError()--
-- yielding 2 --


2

In [32]:
it.close()

-- closing --


# Decorators

The act of altering a function or class object after it has been constructed but before it is bound to its name is called decorating.

In [33]:
def simple_decorator(function):
    print("doing decoration")
    return function

@simple_decorator
def function():
    print("inside function")

doing decoration


In [34]:
function()

inside function


In [35]:
def decorator_with_arguments(arg):
    print("defining the decorator")
    def _decorator(function):
        # arg is available in this inner function
        print("doing decoration with arg %r" % arg)
        return function
    return _decorator
@decorator_with_arguments('mydeco')
def function():
    print('inside function')

defining the decorator
doing decoration with arg 'mydeco'


In [36]:
function()

inside function


In [37]:
def replacing_decorator_with_args(arg):
    print("defining the decorator")
    def _decorator(function):
        # in this inner function, arg is available too
        print("doing decoration, %r" % arg)
        def _wrapper(*args, **kwargs):
            print("inside wrapper, %r %r" % (args, kwargs))
            return function(*args, **kwargs)
        return _wrapper
    return _decorator

@replacing_decorator_with_args("abc")
def function(*args, **kwargs):
    print("inside function, %r %r" % (args, kwargs))
    return 14

defining the decorator
doing decoration, 'abc'


In [38]:
function(11, 12)

inside wrapper, (11, 12) {}
inside function, (11, 12) {}


14

In [39]:
class decorator_class(object):
    def __init__(self, arg):
        # this method is called in the decorator expression
        print("in decorator init, %s" % arg)
        self.arg = arg
    def __call__(self, function):
        # this method is called to do the job
        print("in decorator call, %s" % self.arg)
        return function
    
deco_instance = decorator_class('foo')

in decorator init, foo


In [40]:
@deco_instance
def function(*args, **kwargs):
    print("in function %s %s" % (args, kwargs))

in decorator call, foo


In [41]:
function()

in function () {}
